In [ ]:
"""
@Description :   this script can output *.sdf and *.config for Gazebo with taking a mesh as input
@Author      :   Yan Ding 
@Time        :   2022/12/29 00:46:17
"""

# Import third-party packages

In [ ]:
import os, sys
import shutil
import xml.etree.ElementTree as ET
from google.colab import files
import time

# Utils

In [ ]:
def write(content=None, path=None):
  from google.colab import files
  with open(path, 'w') as f:
    f.write('{}'.format(content))
  # files.download(path)

# Process mesh files

In [ ]:
#@title Compute inertia matrix using online tool  {display-mode: "form", vertical-output: true }

#@markdown <p float="left"> <img src="https://raw.githubusercontent.com/yding25/pic_share/master/mesh_cleaner.png" height="200" /> </p>

#@markdown [Website Link](https://www.hamzamerzic.info/mesh_cleaner/)

#@markdown - Step 1: Upload raw mesh file
#@markdown - Step 2: Download processed mesh file
#@markdown - Step 3: Copy inertia matrix

In [ ]:
#@title Upload processed mesh file (only supports *.dae and *.obj)  {display-mode: "form", vertical-output: true }
uploaded = files.upload()
file_name = []
for item in uploaded.keys():
  print('User uploaded file "{}" with length {} bytes'.format(item, len(uploaded[item])))
  if 'mtl' not in item:
    file_name.append(item)

# Check the uploaded file
signal = False
for item in file_name:
  if '.dae' in item or '.obj' in item:
    signal = True
if not signal:
  print('Error: cannot find mesh file!')
  sys.exit(1)
else:
  pass

Saving box_green.dae to box_green.dae
User uploaded file "box_green.dae" with length 8096 bytes


In [ ]:
#@title Upload computed inertia matrix  {display-mode: "form", vertical-output: true }
#@markdown Please copy inertia matrix and paste it here directly
inertia_matrix = "\u003Cinertial>   \u003C!-- Volume:  1.2500000000e-04 -->   \u003Cmass> 0.1 \u003C/mass>    \u003C!-- Center of mass:  0.0000000000e+00  0.0000000000e+00  0.0000000000e+00 -->   \u003Cpose>  0.0000000000e+00  0.0000000000e+00  0.0000000000e+00 0 0 0 \u003C/pose>    \u003C!-- Inertia matrix -->   \u003Cinertia>     \u003Cixx>  4.1666665040e-05 \u003C/ixx>     \u003Cixy>  0.0000000000e+00 \u003C/ixy>     \u003Cixz>  0.0000000000e+00 \u003C/ixz>     \u003Ciyy>  4.1666665040e-05 \u003C/iyy>     \u003Ciyz>  0.0000000000e+00 \u003C/iyz>     \u003Cizz>  4.1666665040e-05 \u003C/izz>   \u003C/inertia> \u003C/inertial>" #@param {type:"string"}

write(inertia_matrix, 'inertia_matrix.xml')
inertia_tree = ET.parse('inertia_matrix.xml')
inertia_tree_root = inertia_tree.getroot()
pose = None
mass = None
ixx = None
ixy = None
ixz = None
iyy = None
iyz = None
izz = None
for item in inertia_tree_root.iter():
    if item.tag == 'pose':
        pose = item.text
    if item.tag == 'mass':
        mass = item.text
    if item.tag == 'ixx':
        ixx = item.text
    if item.tag == 'ixy':
        ixy = item.text
    if item.tag == 'ixz':
        ixz = item.text
    if item.tag == 'iyy':
        iyy = item.text
    if item.tag == 'iyz':
        iyz = item.text
    if item.tag == 'izz':
        izz = item.text
print('After parsing input inertia matrix:\n')
print(' pose = {}\n mass = {}\n ixx = {}\n ixy = {}\n ixz = {}\n iyy = {}\n iyz = {}\n izz = {}'.format(pose, mass, ixx, ixy, ixz, iyy, iyz, izz))

After parsing input inertia matrix:

 pose =   0.0000000000e+00  0.0000000000e+00  0.0000000000e+00 0 0 0 
 mass =  0.1 
 ixx =   4.1666665040e-05 
 ixy =   0.0000000000e+00 
 ixz =   0.0000000000e+00 
 iyy =   4.1666665040e-05 
 iyz =   0.0000000000e+00 
 izz =   4.1666665040e-05 


In [ ]:
#@title Create output folder  {display-mode: "form", vertical-output: true }
#@markdown Please input your customized gazebo model name, e.g., box_blue
model_name = "box_green" #@param {type:"string"}
if not os.path.exists(model_name):
  os.mkdir(model_name)
  print('the \'{}\' folder has been created'.format(model_name))
else:
  print('the \'{}\' folder already exists'.format(model_name))  

if not os.path.exists(os.path.join(model_name, 'meshes')):
  os.mkdir(os.path.join(model_name, 'meshes'))
  print('the \'{}\' folder has been created'.format(os.path.join(model_name, 'meshes')))
else:
  print('the \'{}\' folder already exists'.format(os.path.join(model_name, 'meshes')))

the 'box_green' folder has been created
the 'box_green/meshes' folder has been created


In [ ]:
#@title Download sample *sdf and *config  {display-mode: "form", vertical-output: true }
if not os.path.exists('model.config') or not os.path.exists('model.sdf'):
  !gdown 1c59M5cvqd_8d-TuV6hSFkgQrFN1Tfgcj
  !gdown 1ZVPt3v06Beetk_lMuPHgfrr6IQp4aIN-
else:
  pass

In [ ]:
#@title Modify sample *config  {display-mode: "form", vertical-output: true }
config_tree = ET.parse('model.config')
config_tree_root = config_tree.getroot()
signal = False # differentiate two "name" in model.config
for item in config_tree_root.iter():
    # print(item)
    if(item.tag == 'name' and not signal):
        signal = True
        item.text = model_name
    if(item.tag == 'description'):
        item.text = 'a model of ' + model_name
print('model.config has been modified successfully.')

model.config has been modified successfully.


In [ ]:
#@title Modify sample *sdf {display-mode: "form", vertical-output: true }
color = "green" #@param ["red", "green", "blue"]
color_value = None
if color == 'red':
  color_value = '1 0 0'
elif color == 'green':
  color_value = '0 1 0'
elif color == 'blue':
  color_value = '0 0 1'
else:
  print('Error: cannot find the color')

sdf_tree = ET.parse('model.sdf')
sdf_tree_root = sdf_tree.getroot()
for item in sdf_tree_root.iter():
    if(item.tag == 'model'):
        item.set('name', model_name)
    if(item.tag == 'uri'):
        item.text = 'model://' + model_name + '/meshes/' + file_name[0]
    if(item.tag == 'pose'):
        item.text = pose
    if(item.tag == 'mass'):
        item.text = mass
    if(item.tag == 'ixx'):
        item.text = ixx
    if(item.tag == 'ixy'):
        item.text = ixy
    if(item.tag == 'ixz'):
        item.text = ixz
    if(item.tag == 'iyy'):
        item.text = iyy
    if(item.tag == 'iyz'):
        item.text = iyz
    if(item.tag == 'izz'):
        item.text = izz
    # set the model color
    if(item.tag == 'ambient'):
        item.text = color_value
    if(item.tag == 'diffuse'):
        item.text = color_value
    if(item.tag == 'specular'):
        item.text = color_value
print('model.sdf has been modified successfully.')

model.sdf has been modified successfully.


In [ ]:
#@title Write modified *config and *sdf {display-mode: "form", vertical-output: true }
config_tree.write(os.path.join(model_name, 'model.config'))
with open(os.path.join(model_name, 'model.config'), 'r+') as filein_config:
  content = filein_config.read()
  filein_config.seek(0, 0)
  filein_config.write('<?xml version="1.0" ?>\n'+content)
filein_config.close()
print('model.config has been writen successfully.')

sdf_tree.write(os.path.join(model_name, 'model.sdf'))
with open(os.path.join(model_name, 'model.sdf'), 'r+') as filein_sdf:
  content = filein_sdf.read()
  filein_sdf.seek(0, 0)
  filein_sdf.write('<?xml version="1.0" ?>\n'+content)
filein_sdf.close()
print('model.sdf has been writen successfully.')

for item in file_name:
  shutil.copyfile(item, os.path.join(model_name, 'meshes', item))

model.config has been writen successfully.
model.sdf has been writen successfully.


In [ ]:
#@title Download *config and *sdf {display-mode: "form", vertical-output: true }
dir_to_zip = model_name
output_filename = model_name + '.zip'
os.system("zip -r {} {}".format(output_filename, dir_to_zip))
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Delete *config and *sdf {display-mode: "form", vertical-output: true }
for file in file_name:
  os.system("rm {}".format(file))
os.system("rm {}".format('inertia_matrix.xml'))
os.system("rm -r {}".format(dir_to_zip))
time.sleep(5)
os.system("rm -r {}".format(output_filename))

0